In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('IMDb - TPM2 IGTI DeCloud') \
    .getOrCreate()

23/04/08 21:21:49 WARN Utils: Your hostname, wedivv-H110M-S2V resolves to a loopback address: 127.0.1.1; using 192.168.1.44 instead (on interface wlp5s0)
23/04/08 21:21:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/08 21:21:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/08 21:21:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
title_basics = spark.read.csv('data/title_basics.tsv', header=True, sep='\t')

In [4]:
title_basics.show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

In [6]:
from pyspark.sql.functions import split

title_basics = title_basics.withColumn("genres_split", split(title_basics["genres"], ","))

In [7]:
from pyspark.sql.functions import explode

title_basics = title_basics.select(["*", explode("genres_split").alias("genre")]).drop("genres_split")


In [8]:
title_basics.show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-----------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|      genre|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-----------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|Documentary|
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|      Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|  Animation|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|      Short|
|tt0000003|    short|      Pauvre 

In [9]:
title_basics.count()

12998329

In [10]:
title_ratings = spark.read.csv('data/title_ratings.tsv', header=True, sep='\t')
title_ratings.show(5)

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.7|    1809|
|tt0000002|          6.0|     233|
|tt0000003|          6.5|    1560|
|tt0000004|          6.1|     152|
|tt0000005|          6.2|    2383|
+---------+-------------+--------+
only showing top 5 rows



In [11]:
title_ratings.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- averageRating: string (nullable = true)
 |-- numVotes: string (nullable = true)



In [12]:
title = title_basics.join(title_ratings, title_basics.tconst == title_ratings.tconst, how='left_outer').drop(title_ratings.tconst)

In [13]:
title.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: string (nullable = true)
 |-- startYear: string (nullable = true)
 |-- endYear: string (nullable = true)
 |-- runtimeMinutes: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- genre: string (nullable = false)
 |-- averageRating: string (nullable = true)
 |-- numVotes: string (nullable = true)



### Qual o gênero de títulos mais frequente?

In [34]:
#title.groupBy('genre').count().orderBy('count', ascending=False).show()
title.groupBy('genre').count().sort('count', ascending=False).show()

+-----------+-------+
|      genre|  count|
+-----------+-------+
|      Drama|2247995|
|     Comedy|1653725|
|      Short|1021850|
|  Talk-Show| 900198|
|Documentary| 764885|
|    Romance| 724729|
|         \N| 643012|
|     Family| 571470|
|       News| 524662|
| Reality-TV| 423455|
|  Animation| 406284|
|      Music| 394008|
|      Crime| 351447|
|     Action| 334580|
|  Adventure| 324325|
|  Game-Show| 252533|
|      Adult| 242704|
|      Sport| 178594|
|    Fantasy| 174119|
|    Mystery| 162448|
+-----------+-------+
only showing top 20 rows



In [31]:
title_type = title.groupBy('titleType').count().sort('count', ascending=False)
title_type.show()


+------------+-------+
|   titleType|  count|
+------------+-------+
|   tvEpisode|9297133|
|       short|1692723|
|       movie| 862338|
|       video| 530560|
|    tvSeries| 286685|
|     tvMovie| 169820|
|tvMiniSeries|  53027|
|   videoGame|  45227|
|   tvSpecial|  39804|
|     tvShort|  21009|
|     tvPilot|      1|
| radioSeries|      1|
|radioEpisode|      1|
+------------+-------+



### Quantos filmes (incluindo os da televisão) foram lançados no ano de 2015?

In [41]:
from pyspark.sql.functions import countDistinct

movies_2015 = title.filter(((title.titleType == 'movie') | (title.titleType == 'tvMovie')) & (title.startYear == '2015'))
movies_2015 = movies_2015.select(countDistinct('tconst').alias('filmes lançados em 2015'))
movies_2015.show()


+-----------------------+
|filmes lançados em 2015|
+-----------------------+
|                  19987|
+-----------------------+



### Qual o gênero com a melhor nota média de títulos?

In [16]:
from pyspark.sql.functions import avg, desc

# Agrupa por gênero e calcula a média de averageRating
grouped = title.groupBy('genre').agg(avg('averageRating').alias('avg_rating'))

# Ordena pelo valor médio da nota em ordem decrescente
sorted_grouped = grouped.sort(desc('avg_rating'))

sorted_grouped.show()


23/04/07 14:49:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/07 14:49:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/07 14:49:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/07 14:49:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/07 14:49:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/07 14:49:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/07 14:49:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/07 14:49:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+-----------+-----------------+
|      genre|       avg_rating|
+-----------+-----------------+
|    History|7.353780102645096|
|Documentary|7.240198535554619|
|  Biography|7.175531914893612|
|    Mystery|7.170086406897958|
|      Crime|7.159842868485991|
|  Adventure|7.107629703351799|
|    Fantasy|7.095145650845389|
|  Animation|7.089381171483267|
|    Western|7.080683426568721|
|     Family|7.070054926034511|
|      Drama|7.040979155040105|
|        War|7.009115134414917|
|     Action| 7.00709813874791|
|      Sport|6.966792418526419|
|     Comedy|6.960016550918294|
|      Music|6.927469624015723|
| Reality-TV|6.892611170895987|
|  Game-Show|6.876828101904179|
|    Romance|6.864016164703988|
|      Short|6.791292438368553|
+-----------+-----------------+
only showing top 20 rows



### Qual o vídeo game do gênero aventura mais bem avaliado em 2020?

In [38]:
game_adventure_2020 = title.filter((title.titleType == 'videoGame') & (title.startYear == '2020') & (title.genre == 'Adventure'))

game_adventure_2020 = game_adventure_2020.select(['primaryTitle', 'averageRating', 'tconst', 'genre']).distinct()

titles_games = ['Omori', 'Final Fantasy VII Remake', 'Half-Life: Alyx', 'Ghost of Tsushima']

matching_rows = game_adventure_2020.filter(game_adventure_2020.primaryTitle.isin(titles_games))

matching_rows.show()

+--------------------+-------------+----------+---------+
|        primaryTitle|averageRating|    tconst|    genre|
+--------------------+-------------+----------+---------+
|               Omori|          9.2|tt14106780|Adventure|
|   Ghost of Tsushima|          9.3| tt7651352|Adventure|
|     Half-Life: Alyx|          9.5|tt11321196|Adventure|
|Final Fantasy VII...|          9.1| tt6057416|Adventure|
+--------------------+-------------+----------+---------+



### Quantos títulos de filmes diferentes existem?

In [18]:
mv_titles = title.select(countDistinct('primaryTitle').alias('Titulos de filmes'))
mv_titles.show()

+-----------------+
|Titulos de filmes|
+-----------------+
|          3931660|
+-----------------+



### Qual a duração média dos filmes com conteúdo adulto?

In [ ]:
adult_movie = title.filter(title.genre == 'Adult')
adult_movie = adult_movie.select('tconst', 'runtimeMinutes').dropDuplicates(['tconst'])


adult_movie.describe().show()


+-------+---------+------------------+
|summary|   tconst|    runtimeMinutes|
+-------+---------+------------------+
|  count|   242704|            242704|
|   mean|     null|  93.6222312036195|
| stddev|     null|57.459346444443966|
|    min|tt0062727|                 1|
|    max|tt9916266|                \N|
+-------+---------+------------------+



### Quantos filmes têm o título atual (“primary”) diferente do título original? 

In [65]:
diff_title = title.filter(title.primaryTitle != title.originalTitle)
diff_count = diff_title.agg(countDistinct('tconst').alias('count'))

diff_count.show()


+------+
| count|
+------+
|125046|
+------+



### Qual o filme que tem o nome mais longo?

In [60]:
from pyspark.sql.functions import length

longest_title = title.select('tconst', 'primaryTitle', length('primaryTitle').alias('title_length'))\
    .sort(desc('title_length'))

longest_title.show()


+----------+--------------------+------------+
|    tconst|        primaryTitle|title_length|
+----------+--------------------+------------+
|tt12985206|Otoko wa chi _ ko...|         419|
|tt13007080|Creampie Voluntee...|         409|
| tt7727908|Yufukuna kurashi ...|         408|
| tt7727908|Yufukuna kurashi ...|         408|
|tt12870546|An Ultra K-Cup Ti...|         405|
|tt12866892|I'm A Dirty Old M...|         401|
| tt7395478|'Oppai dakenara s...|         400|
|tt13558536|While Horsing Aro...|         386|
|tt13053838|The Dumb Cunt Is ...|         383|
|tt13931914|EE.UU. Se replieg...|         368|
| tt7509356|Kanojo ni furaret...|         366|
|tt13278010|This Son Was Secr...|         351|
|tt10813978|Y- the total ease...|         347|
|tt10813978|Y- the total ease...|         347|
|tt10813978|Y- the total ease...|         347|
|tt13009172|I'm the Silent Ty...|         340|
|tt14128648|'No, No, No! Your...|         339|
|tt13356578|An Up Close and P...|         337|
| tt7298012|O

### Qual filme tem a maior quantidade de votos? 

In [53]:
films_list = ['tt0110613', 'tt0111161', 'tt0110557', 'tt0110570']

movies_hightest = title.select('tconst', 'averageRating')
movie_matching = movies_hightest.filter(movies_hightest.tconst.isin(films_list))
movie_matching = movie_matching.select('tconst', 'averageRating')

movie_matching.describe().show()




+-------+---------+------------------+
|summary|   tconst|     averageRating|
+-------+---------+------------------+
|  count|       10|                10|
|   mean|     null| 6.329999999999998|
| stddev|     null|1.4384018909887462|
|    min|tt0110557|               5.3|
|    max|tt0111161|               9.3|
+-------+---------+------------------+



### Qual é a menor nota média de um filme? 

In [45]:
movies_lowest = title.filter(title.titleType == 'movie')
movies_lowest = movies_lowest.select('averageRating')
movies_lowest.describe().show()


+-------+------------------+
|summary|     averageRating|
+-------+------------------+
|  count|            461313|
|   mean|  6.10826271967178|
| stddev|1.3100258236943179|
|    min|               1.0|
|    max|               9.9|
+-------+------------------+

